The purpose of this notebook is to combine data from our 2 data sources.

Specifically look at the following 2 webpages, with different data on the same election:
- https://www.electionsireland.org/result.cfm?election=1977&cons=219
- https://www.irelandelection.com/election.php?elecid=11&electype=1&constitid=48

Note: The quotas are different.

For every elections we want
- Number of Constituencies
- How many consituency do we have vote data on?
- What was the quota?
- What was the votes/quota in first count?
- What was the lowest votes/quota?
- What was the highest votes/quota?
- Who transfered to who (if you have transfer data)

What parties do well in what areas in the last 25 years? 
eg total town council, dail seats, and other have been elected for a given area. eg sf:3 ff:16 fg: 14 

In [1]:
import pandas as pd
import numpy as np

KeyboardInterrupt: 

### Introducing the first data set

The first dataset comes from https://www.electionsireland.org/ 

This data has date 12 columns:

- date: _str_, This is sometimes the exact date of the election, othertimes its just the year, other times its the year and the month: needs to be combined with the other data set which has the year.

- election_type: _str_, Is the type of election the candidate was running in, eg Local, Seanad, Dail, Bi-Election, This data needs to cleaned because it also includes intra-party elections and resignations. (Interesting data but not needed for this analysis)

- party: _str_, The party a candidate ran with, or for independents, it lists them as belonging to a party called independent.

- status: _str_, If they were elected or not. This data needs to be cleaned as it includes other information such as whether the candidate made the cutoff for expenses. 

- constituency_name: _str_, The name of the constituency the candidate ran in.

- seat: _int_, Contains the order in which the candidate was elected, if the candidate wasnt elected then its None.

- count_eliminated: _int_, Which count the candidate was either **elected** or **lost**. 

- first_pref_count _int_, The count of first preference votes recieved. 

- first_pref_pct: _float_, The % of all first preference votes a candidate recieved. 

- pct_of_quota_reached_with_first_pref: _float_, The % of the quota that a candidate reached with first preference votes.

- ran_unopposed: _bool_, True if the candidate ran unopposed (more common in earlier elections)

- candidate: _str_, The candidate's name

- candidate_ID: _int_, A unique ID for each candidate 


In [ ]:
df1 = pd.read_parquet('electionsireland_data/ElectionsIreland_candidate.parquet')
df1 = df1.rename(columns={'ID':'candidate_ID'})
print(df1.columns)
print(df1.shape)
df1.head()

Index(['date', 'election_type', 'party', 'status', 'constituency_name', 'seat',
       'count_eliminated', 'first_pref_count', 'first_pref_pct',
       'pct_of_quota_reached_with_first_pref', 'ran_unopposed', 'candidate',
       'candidate_ID'],
      dtype='object')
(30070, 13)


,date,election_type,party,status,constituency_name,seat,count_eliminated,first_pref_count,first_pref_pct,pct_of_quota_reached_with_first_pref,ran_unopposed,candidate,candidate_ID
0,23 June 1960,By Election,Labour,Not Elected,Carlow Kilkenny,None,NaN,7678.0,0.2016,NaN,False,Seamus Pattison,1
1,1961,17th Dail,Labour,Elected,Carlow Kilkenny,4,NaN,4116.0,0.0954,NaN,False,Seamus Pattison,1
2,1965,18th Dail,Labour,Elected,Carlow Kilkenny,3,NaN,6299.0,0.1408,NaN,False,Seamus Pattison,1
3,1969,19th Dail,Labour,Elected,Carlow Kilkenny,4,NaN,6041.0,0.1311,NaN,False,Seamus Pattison,1
4,1973,20th Dail,Labour,Elected,Carlow Kilkenny,4,NaN,5300.0,0.1134,NaN,False,Seamus Pattison,1


### Introducing the second dataset

The first dataset comes from https://www.irelandelection.com/ 

This data has date 10 columns:

- election: _str_, Contains a string describing the election usually just the consitituency and the date of the election

- elected: _bool_, If they were elected or not. This data needs to be combined with Status from the first dataframe. 

- party: _str_, The party a candidate ran with

- first_pref_pct: _float_, The % of all first preference votes a candidate recieved. 

- first_pref_count _int_, The count of first preference votes recieved. 

- first_pref_quota_ratio: _float_, The % of the quota that a candidate reached with first preference votes.

- year: _int_, The year of the election. *NB:* Sometimes there is two elections of the same kind in the same year, (eg 2 dail elections in 1982).

- election_type: _str_, Is the type of election the candidate was running in, eg Local, Seanad, Dail, Bi-Election.

- candidate: _str_, The candidate's name. Unfortunatly there is 2 candidates have the same, they are essentially grouped as the same person, we fix this by linking records with the first dataset. 

- constituency: _str_, The name of the constituency the candidate ran in.

- election_type: _str_, Is the type of election the candidate was running in, eg Local, Dail, Bi-Election.

In [ ]:
df2 = pd.read_parquet('irelandelection/ALL_CANDIDATES.parquet')
print(df2.columns)
print(df2.shape)

df2.head()

Index(['election', 'elected', 'party', 'first_pref_pct', 'first_pref_count',
       'first_pref_quota_ratio', 'year', 'candidate', 'constituency',
       'election_type'],
      dtype='object')
(36243, 10)


,election,elected,party,first_pref_pct,first_pref_count,first_pref_quota_ratio,year,candidate,constituency,election_type
0,2004 Local Election - Thomastown,True,Labour Party,0.085,641,0.51,2004,Ann Phelan,Thomastown,LOCAL
1,2009 Local Election - Thomastown,True,Labour Party,0.156,1183,0.78,2009,Ann Phelan,Thomastown,LOCAL
2,2011 general election - Carlow–Kilkenny,True,Labour Party,0.109,8072,0.66,2011,Ann Phelan,Carlow–Kilkenny,GENERAL
3,2016 general election - Carlow–Kilkenny,False,Labour Party,0.063,4391,0.38,2016,Ann Phelan,Carlow–Kilkenny,GENERAL
0,1982 (Feb) general election - Carlow–Kilkenny,False,Fianna Fáil,0.017,907,0.10,1982,John McGuinness,Carlow–Kilkenny,GENERAL


### Cleaning Dataframe 1

So we need to clean the following columns in dataframe 1:
- seat and count_eliminated can be dropped as for this analysis we dont really care about the order in which candidates were elected. 
- date, this needs to be converted to a year columns.
- election_type, this should read the same as dataframe 2 with options like GENERAL, LOCAL and SEANAD
- status, will be kept, but we want a column that is a simple bool TRUE or FALSE was the candidate elected.


In [ ]:
# dropping seat and count_eliminated

df1 = df1.drop(columns=['seat','count_eliminated'])
df1.head(3)

,date,election_type,party,status,constituency_name,first_pref_count,first_pref_pct,pct_of_quota_reached_with_first_pref,ran_unopposed,candidate,candidate_ID
0,23 June 1960,By Election,Labour,Not Elected,Carlow Kilkenny,7678.0,0.2016,NaN,False,Seamus Pattison,1
1,1961,17th Dail,Labour,Elected,Carlow Kilkenny,4116.0,0.0954,NaN,False,Seamus Pattison,1
2,1965,18th Dail,Labour,Elected,Carlow Kilkenny,6299.0,0.1408,NaN,False,Seamus Pattison,1


In [ ]:
#cleaning date

def get_year_from_date_string(date_str):
    if date_str == None:
        return 0
    elif len(date_str) > 4:
        try:
            return int(date_str[-4:])
        except:
            if isinstance(date_str[-1],int):#last letter is an int
                return date_str[-4:]
    else:
        return int(date_str)

df1 = df1.reset_index().drop(columns=['index'])
df1['year'] = df1.date.apply(get_year_from_date_string)
df1 = df1.drop(columns=['date'])

df1.head(3)

,election_type,party,status,constituency_name,first_pref_count,first_pref_pct,pct_of_quota_reached_with_first_pref,ran_unopposed,candidate,candidate_ID,year
0,By Election,Labour,Not Elected,Carlow Kilkenny,7678.0,0.2016,NaN,False,Seamus Pattison,1,1960.0
1,17th Dail,Labour,Elected,Carlow Kilkenny,4116.0,0.0954,NaN,False,Seamus Pattison,1,1961.0
2,18th Dail,Labour,Elected,Carlow Kilkenny,6299.0,0.1408,NaN,False,Seamus Pattison,1,1965.0


In [ ]:
#cleaning election_type

def get_election_type_from_string(election_type_str):
    if 'Town' in election_type_str or 'Local' in election_type_str :
        return 'LOCAL'
    elif 'Dail' in election_type_str:
        return 'GENERAL'
    elif 'Seanad' in election_type_str:
        return 'SEANAD'
    elif 'Westminster' in election_type_str:
        return 'Westminster'.upper()
    elif 'European' in election_type_str:
        return 'EUROPEAN'
    elif 'By Election' in election_type_str:
        return 'BI-ELECTION'
    else: # for the rows that represent resignations or appointments or some other event in a politicans career
        return None

df1['election_type'] = df1.election_type.apply(get_election_type_from_string)
print(df1.election_type.unique())

['BI-ELECTION' 'GENERAL' 'EUROPEAN' None 'LOCAL' 'SEANAD' 'WESTMINSTER']


In [ ]:
df1.status.unique()

array(['Not Elected', 'Elected', 'Appointed', 'Resigned', 'Disqualified',
       None, 'Co-opted', 'Candidate', 'Dublin', 'Lucan',
       'awaiting update', 'changed\xa0to',
       'Dublin University (Trinity College)', 'Died in office:',
       'Agricultural Panel', 'Gorey\xa0\xa0\xa0-\xa0\xa0\xa0Resigned',
       'Nominated by Taoiseach', 'Inishowen', '(Replaced  David McKenna)',
       '(Replaced  Michael Flynn)', 'Administrative Panel', 'Drumcliff',
       'Ballybrack\xa0\xa0\xa0-\xa0\xa0\xa0Resigned\n            \n              (ill health)',
       'Kilkenny\xa0\xa0\xa0-\xa0\xa0\xa0Resigned\n            \n              (dual mandate TD)'],
      dtype=object)

In [ ]:
# cleaning status
def was_elected(status):
    status = str(status) # some of the status are None 

    if status == 'Elected':
        return True
    elif status == 'Not Elected':
        return False

    else: # for the rows that represent resignations or appointments or some other event in a politicans career
        return None

df1['elected'] = df1.status.apply(was_elected)
df1.head()

,election_type,party,status,constituency_name,first_pref_count,first_pref_pct,pct_of_quota_reached_with_first_pref,ran_unopposed,candidate,candidate_ID,year,elected
0,BI-ELECTION,Labour,Not Elected,Carlow Kilkenny,7678.0,0.2016,NaN,False,Seamus Pattison,1,1960.0,False
1,GENERAL,Labour,Elected,Carlow Kilkenny,4116.0,0.0954,NaN,False,Seamus Pattison,1,1961.0,True
2,GENERAL,Labour,Elected,Carlow Kilkenny,6299.0,0.1408,NaN,False,Seamus Pattison,1,1965.0,True
3,GENERAL,Labour,Elected,Carlow Kilkenny,6041.0,0.1311,NaN,False,Seamus Pattison,1,1969.0,True
4,GENERAL,Labour,Elected,Carlow Kilkenny,5300.0,0.1134,NaN,False,Seamus Pattison,1,1973.0,True


### Cleaing dataframe 2

Renaming:
Dataframe 2 has a column called ```first_pref_quota_ratio``` which is the same information as ```pct_of_quota_reached_with_first_pref``` in dataframe 1

In [ ]:
df2 = df2.rename(columns={'first_pref_quota_ratio':'pct_of_quota_reached_with_first_pref','constituency':'constituency_name'}).reset_index().drop(columns=['index'])

print(df2.columns)
print(df2.shape)

Index(['election', 'elected', 'party', 'first_pref_pct', 'first_pref_count',
       'pct_of_quota_reached_with_first_pref', 'year', 'candidate',
       'constituency_name', 'election_type'],
      dtype='object')
(36243, 10)


### Fixing Constitenuecy names.

I noticed that the constituency names are slightly different

In [ ]:
set(df1[df1['election_type'] =='LOCAL'].constituency_name.unique()).difference(set(df2[df2['election_type'] =='LOCAL'].constituency_name.unique()))
# I dont bother to fix this but we probably should get a list of all constituency names and the map them onto whatever the website calls them. 
# this is easy with dail elections but not so easy with local elections as i cant find anything about it

{'Adare Rathkeale',
 'Artane Whitehall',
 'Athenry Oranmore',
 'Bailieborough Cootehill',
 'Ballaghadereen',
 'Ballincollig Carrigaline',
 'Ballybay',
 'Ballybay Clones',
 'Ballyfermot Drimnagh',
 'Ballymote Tobercurry',
 'Ballymun Finglas',
 'Ballymun Whitehall',
 'Ballyshannon',
 'Bandon Kinsale',
 'Bantry West Cork',
 'Beaumont Donaghmede',
 'Belturbet TC',
 'Blanchardstown Mulhuddart',
 'Blarney Macroom',
 'Borris-in-Ossory Mountmellick',
 'Bray No 1',
 'Bray No 2',
 'Bray No 3',
 'Bray South',
 'Bundoran',
 'Cabra Finglas',
 'Cabra Glasnevin',
 'Callan Thomastown',
 'Cappaghmore Kilmallock',
 'Carlow No 1',
 'Carlow No 2',
 'Carrick on Suir',
 'Carrick-On-Shannon',
 'Carrickmacross Castleblaney',
 'Cashel Tipperary',
 'Cavan Belturbet',
 'Celbridge Leixlip',
 'Clare West',
 'Clonakilty',
 'Conamara',
 'Cootehill',
 'Cork East',
 'Cork West',
 'Crumlin Kimmage',
 'Drogheda No 1 Laurence Gate',
 'Drogheda No 2 West Gate',
 'Drogheda No 3 Duleek Gate',
 'Droichead Nua',
 'Dromahaire'

In [ ]:
set(df1[df1['election_type'] =='GENERAL'].constituency_name.unique()).difference(set(df2[df2['election_type'] =='GENERAL'].constituency_name.unique()))

{'Antrim East',
 'Antrim Mid',
 'Antrim North',
 'Antrim South',
 'Armagh Mid',
 'Armagh North',
 'Armagh South',
 'Athlone Longford',
 'Belfast Cromac',
 'Belfast Duncairn',
 'Belfast Falls',
 'Belfast Ormeau',
 'Belfast Pottinger',
 'Belfast Shankill',
 "Belfast St Anne's",
 'Belfast Victoria',
 'Belfast Woodvale',
 'Carlow',
 'Carlow Kildare',
 'Carlow Kilkenny',
 'Cavan East',
 'Cavan Monaghan',
 'Cavan West',
 'Clare East',
 'Clare Galway South',
 'Clare West',
 'Cork',
 'Cork City North',
 'Cork City South',
 'Cork East and North East',
 'Cork Mid/North/South/South East and West',
 'Cork North Central',
 'Cork North East',
 'Cork North West',
 'Cork South Central',
 'Cork South East',
 'Cork South West',
 'Donegal Leitrim',
 'Donegal North',
 'Donegal North East',
 'Donegal South',
 'Donegal South West',
 'Down East',
 'Down Mid',
 'Down North',
 'Down South',
 'Down West',
 'Dublin College Green',
 'Dublin Harbour',
 'Dublin Mid',
 'Dublin Mid West',
 'Dublin North Central',
 'D

In [ ]:
df1.index

RangeIndex(start=0, stop=30070, step=1)

In [ ]:
current_constituencies = pd.read_html('https://en.wikipedia.org/wiki/D%C3%A1il_constituencies',flavor='bs4')[3]
historic_constituencies = pd.read_html('https://en.wikipedia.org/wiki/Historic_D%C3%A1il_constituencies',flavor='bs4')[1]
historic_constituencies

,Constituency,County or city,Created,Abolished,Seats
0,Antrim,Antrim,1921,1922.0,7
1,Antrim East,Antrim,1918,1921.0,1
2,Antrim Mid,Antrim,1918,1921.0,1
3,Antrim North,Antrim,1918,1921.0,1
4,Antrim South,Antrim,1918,1921.0,1
...,...,...,...,...,...
255,Wexford North,Wexford,1918,1921.0,1
256,Wexford South,Wexford,1918,1921.0,1
257,Wicklow[aq],Wicklow,1923,NaN,345
258,Wicklow East,Wicklow,1918,1921.0,1


In [ ]:
current_constituencies

,Constituency,Seats
0,Carlow–Kilkenny,5
1,Cavan–Monaghan,5
2,Clare,4
3,Cork East,4
4,Cork North-Central,4
5,Cork North-West,3
6,Cork South-Central,4
7,Cork South-West,3
8,Donegal,5
9,Dublin Bay North,5


# delete the above which is in Matthew's code.

### Joining the two datasets:

Some of the data in Dataframe 1 is missings, hence why we are combining these 2 dataframes.
I remove entries from dataframe 1 that arent elected. 

Recall that some rows in dataframe 1 just represent a politician being appointed as minister or resigning, not always an election.

In [ ]:
df1 = df1[~df1.elected.isnull()].reset_index()\
    .drop(columns=['index'])

df1

,election_type,party,status,constituency_name,first_pref_count,first_pref_pct,pct_of_quota_reached_with_first_pref,ran_unopposed,candidate,candidate_ID,year,elected
0,BI-ELECTION,Labour,Not Elected,Carlow Kilkenny,7678.0,0.2016,NaN,False,Seamus Pattison,1,1960.0,False
1,GENERAL,Labour,Elected,Carlow Kilkenny,4116.0,0.0954,NaN,False,Seamus Pattison,1,1961.0,True
2,GENERAL,Labour,Elected,Carlow Kilkenny,6299.0,0.1408,NaN,False,Seamus Pattison,1,1965.0,True
3,GENERAL,Labour,Elected,Carlow Kilkenny,6041.0,0.1311,NaN,False,Seamus Pattison,1,1969.0,True
4,GENERAL,Labour,Elected,Carlow Kilkenny,5300.0,0.1134,NaN,False,Seamus Pattison,1,1973.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...
28585,WESTMINSTER,Democratic Unionist,Not Elected,Newry and Armagh,5764.0,0.1284,NaN,False,William Irwin,4203,2010.0,False
28586,WESTMINSTER,Democratic Unionist,Not Elected,Newry and Armagh,13177.0,0.2459,NaN,False,William Irwin,4203,2017.0,False
28587,WESTMINSTER,Democratic Unionist,Not Elected,Newry and Armagh,11000.0,0.2166,NaN,False,William Irwin,4203,2019.0,False
28588,BI-ELECTION,Non party/Independent,Not Elected,Cork South Central,219.0,0.0052,0.01,False,Brian McEnery,4211,1994.0,False


In [ ]:
print(df1.shape,df2.shape)
print(sorted(set(df1.columns)))
print(sorted(set(df2.columns)))

(28590, 12) (36243, 10)
['candidate', 'candidate_ID', 'constituency_name', 'elected', 'election_type', 'first_pref_count', 'first_pref_pct', 'party', 'pct_of_quota_reached_with_first_pref', 'ran_unopposed', 'status', 'year']
['candidate', 'constituency_name', 'elected', 'election', 'election_type', 'first_pref_count', 'first_pref_pct', 'party', 'pct_of_quota_reached_with_first_pref', 'year']


In [ ]:
#df1.year.unique()
df1 = df1.reindex(columns=[
    'year',
    'candidate',
    'candidate_ID',
    'constituency_name',
    'party',
    'elected',
    'election_type',
    'first_pref_count',
    'first_pref_pct',
    'pct_of_quota_reached_with_first_pref',
    'ran_unopposed',
    'status',
    ])
df1.head(3)

,year,candidate,candidate_ID,constituency_name,party,elected,election_type,first_pref_count,first_pref_pct,pct_of_quota_reached_with_first_pref,ran_unopposed,status
0,1960.0,Seamus Pattison,1,Carlow Kilkenny,Labour,False,BI-ELECTION,7678.0,0.2016,NaN,False,Not Elected
1,1961.0,Seamus Pattison,1,Carlow Kilkenny,Labour,True,GENERAL,4116.0,0.0954,NaN,False,Elected
2,1965.0,Seamus Pattison,1,Carlow Kilkenny,Labour,True,GENERAL,6299.0,0.1408,NaN,False,Elected


In [ ]:
df2.year = df2.year.astype(float)
df2.year.unique()

array([2004., 2009., 2011., 2016., 1982., 1985., 1991., 1997., 1999.,
       2002., 2007., 2020., 2014., 2015., 1987., 1989., 1992., 1960.,
       1961., 1965., 1967., 1969., 1973., 1974., 1977., 1979., 1981.,
       1994., 1934., 1950., 1955., 1948., 1951., 1954., 1957., 1956.,
       1925., 1932., 1933., 1937., 1938., 1943., 1944., 1923., 1927.,
       1921., 1922., 1984., 2019., 1968., 1959., 1942., 1945., 1928.,
       1980., 1953., 1924., 1998., 1996., 1983., 1995., 2013., 1972.,
       2010., 1970., 1949., 1958., 1946., 1929., 1926., 1963., 1952.,
       1976., 1947., 1964., 1975., 1935., 1966., 2005., 1920., 1930.,
       2001., 2000., 1936., 1931., 1939., 1940.])

In [ ]:
#pd.concat([df1,df2],axis=1).sort_values('year')

In [ ]:
print(set(df2.columns))
print(set(df1.columns))
print({'election',  'status',  'ran_unopposed'})

{'first_pref_pct', 'election_type', 'constituency_name', 'elected', 'candidate', 'election', 'first_pref_count', 'year', 'party', 'pct_of_quota_reached_with_first_pref'}
{'first_pref_pct', 'ran_unopposed', 'status', 'candidate_ID', 'election_type', 'constituency_name', 'elected', 'pct_of_quota_reached_with_first_pref', 'candidate', 'year', 'party', 'first_pref_count'}
{'ran_unopposed', 'status', 'election'}


In [ ]:
import jellyfish

def are_names_the_similar(name_1,name_2):
    if name_1==name_2:
        return True
    elif jellyfish.levenshtein_distance(name_1,name_2) <=2:
        return True
    else:
        return False

get the similar names
then find a match where both tables have a record, eg mary b and marie b lost a general election in the same constitency in 1969. 
then we can map mary b and marie b back to the same ID
then concat the two tables and anywhere where we see 

CASE 1: We the same rows in each table
- Case 1A: Data is the same
- Case 1B: Name is similar but one bit of data is different, eg pct_of_quota_reached_with_first_pref is nan in one table and 0.38 in another.

CASE 2: We have data in one table that is not in the other, eg election results for the 70s


What do we want from this:
- I want every name to be linked with an ID
- We want a table that has all the election results, with no duplication, and with the most accurate information. 

We know that one table already has an ID. 

In [ ]:
all_names_without_ID = set(df2.candidate.unique())
name_set = {}
for name_1 in df1.candidate.unique():
    name_set[name_1] = []
    for name_2 in all_names_without_ID:
        if are_names_the_similar(name_1,name_2):
            name_set[name_1].append(name_2)
            print(name_1,name_2)
            

    #print(similar_names)   
    # temp_df = df1[df1.year == year]
    # temp_df2 = df2[df2.year == year]
    # for name in temp_df.candidate:
    #     mp_df2.candidate.apply(lambda name_2: jellyfish.levenstein_distance)

    # print(temp_df.head())
    # print(temp_df2.head())
    # break

Seamus Pattison Séamus Pattison
John McGuinness John McGuinness
Phil Hogan Paul Hogan
Phil Hogan Phil Hogan
M J Nolan M.J. Nolan
M J Nolan M. J. Nolan
Jim Townsend Jim Townsend
Mary White Mary White
Fergal Browne Fergal Browne
Eddie Collins Hughes Eddie Collins Hughes
Billy Nolan Billy Nolan
Billy Nolan Bill Nolan
Mary M White Mary White
Tommy Kinsella Tommy Kinsella
Jimmy Brennan Jimmy Brennan
Jimmy Brennan Tommy Brennan
Walter Lacey Walter Lacey
Geraldine Callinan O'Dea Geraldine Callinan-O'Dea
Annie Parker Byrne Annie Parker-Byrne
Arthur McDonald Arthur McDonald
Enda Nolan Enda Nolan
Francis Deane Francis Deane
Francis Deane Francis Dunne
Noel Kennedy Noel Kennedy
Noel Kennedy Ned Kennedy
Clifford Kelly Clifford Kelly
Michael McCarey Michael McCarthy
Michael McCarey Michael McCartney
Michael McCarey Michael Carey
Michael McCarey Michael McCarey
Matt McPhillips Matt McPhillips
Joe O'Reilly Joe Reilly
Joe O'Reilly Joe O'Neill
Joe O'Reilly Tom O'Reilly
Joe O'Reilly Joe O'Reilly
Joe O'R

loop through and see if the name is the same or similar, are there records in both tables.

In [ ]:
df1 = df1.fillna(value='s;alfjfdwao;fsl;').replace('s;alfjfdwao;fsl;',None)
df2 = df2.fillna(value='s;alfjfdwao;fsl;').replace('s;alfjfdwao;fsl;',None)

In [ ]:
parties = set(df1.party).union(set(df2.party))
already_added = []
party_dict = {}

for party in sorted(parties):
    party_dict[party] = []
    if party not in already_added:
        for party_ in parties:
            if jellyfish.jaro_winkler(party,party_) > 0.9:
                party_dict[party].append(party_)
                already_added.append(party_)

party_dict

{'': [],
 'Ailtirí na hAiséirghe': ['Ailtirí na hAiséirghe'],
 'Alliance': ['Alliance'],
 'An Chomhdháil Phobail': ['An Chomhdháil Phobail'],
 'Anglo Irish Agreement': ['Anglo Irish Agreement'],
 'Anti H-Block': ['Anti H-Block'],
 'Anti-Austerity Alliance': ['Anti-Austerity Alliance'],
 'Anti-Treaty Sinn Féin': ['Anti-Treaty Sinn Féin'],
 'Aontacht Eireann (Irish Unity)': ['Aontacht Eireann (Irish Unity)'],
 'Aontacht Éireann': ['Aontacht Éireann'],
 'Aontú': ['Aontú'],
 "Businessmen's Party": ["Businessmen's Party"],
 'Cannabis is Safer than Alcohol': ['Cannabis is Safer than Alcohol'],
 'Catholic Democrats': ['Catholic Democrats'],
 'Ceann Comhairle': ['Ceann Comhairle', 'Ceann Comhairle (Speaker)'],
 'Ceann Comhairle (Speaker)': [],
 'Chamber of Commerce': ['Chamber of Commerce'],
 'Chamber of Commerce and Ratepayers Association': ['Chamber of Commerce and Ratepayers Association'],
 'Christian Centrist': ['Christian Centrist'],
 'Christian Solidarity': ['Christian Solidarity'],
 'Cl

In [ ]:
def create_new_dictionary(dict1,dict2):
    new_dict = {}
    new_dict['df1_index'] = dict1.get('Index')
    new_dict['df2_index'] = dict2.get('Index')
    if dict1.get('candidate') != dict2.get('candidate'): # if names arent the exact same
        new_dict['AKA'] = dict2.get('candidate') if len(dict2.get('candidate')) > len(dict1.get('candidate')) else dict1.get('candidate')

    keys = set(dict1.keys()).union(set(dict2.keys()))
    keys.remove('Index')
    keys.remove('candidate')
    for key in keys:
        if dict1.get(key) == None:
            new_dict[key] = dict2.get(key) 
            
        elif dict2.get(key) == None:
            new_dict[key] = dict1.get(key)
        else:
            if len(str(dict2.get(key))) > len(str(dict1.get(key))): # we pick the longest values because on floats that means higher precision and on constituency names it means more detail
                new_dict[key] = dict2.get(key)
            else:
                new_dict[key] = dict1.get(key)
                
    return new_dict

In [ ]:
jellyfish.jaro_winkler('Labour Party','Labour')

0.9

In [ ]:
df2['candidate_ID'] = ''

combined_dataframe_dicts = []
for name_from_df1,list_of_names_from_df2 in name_set.items():
    temp_1 = df1[df1.candidate.isin([name_from_df1])]
    temp_2 = df2[df2.candidate.isin(list_of_names_from_df2)]
    for row_df1 in temp_1.itertuples():
        for row_df2 in temp_2.itertuples():
            if row_df2.year == row_df1.year \
                and row_df2.election_type == row_df1.election_type\
                    and jellyfish.jaro_winkler(str(row_df1.constituency_name),str(row_df2.constituency_name)) > 0.8\
                or row_df2.year == row_df1.year \
                    and row_df2.election_type == row_df1.election_type\
                        and jellyfish.jaro_winkler(str(row_df1.party),str(row_df2.party)) > 0.8\
                or row_df2.year == row_df1.year \
                    and jellyfish.jaro_winkler(str(row_df1.party),str(row_df2.party)) > 0.8 \
                        and jellyfish.jaro_winkler(str(row_df1.constituency_name),str(row_df2.constituency_name)) > 0.8:
                
                # theyre the same if they are the same type of election, in the same year, and the constituency names are similar. 
                # The only exception i can think of this would be two men called tomas o brien and thomas o brien who run in local elections, one in kildare east and another in kildare west and theyre in similar sounding parties like Marxist Sinn Fein and Sinn Fein
                #print('beep')
                
                df2.loc[row_df2.Index,'candidate_ID'] = row_df1.candidate_ID
                d = create_new_dictionary(row_df1._asdict(),row_df2._asdict())
                combined_dataframe_dicts.append(d)
                
            # except Exception as e:
            #     print(str(e))
            #     print(row_df1)
            #     print(row_df2)
            #     print('-------------')

In [ ]:
jellyfish.jaro_winkler(str('Fianna Fáil'),str('Fianna Fail'))

0.9636363636363636

Now I need to find the ones that dont have an ID associated with them, and assign them an ID

In [ ]:
combined_dataframe = pd.DataFrame(combined_dataframe_dicts)
combined_dataframe

,df1_index,df2_index,AKA,ran_unopposed,elected,first_pref_count,election_type,year,party,first_pref_pct,status,candidate_ID,constituency_name,election,pct_of_quota_reached_with_first_pref
0,0,72,Seamus Pattison,False,False,7678.0,BI-ELECTION,1960.0,Labour Party,0.2020,Not Elected,1,Carlow Kilkenny,1960 by-election - Carlow–Kilkenny,0.40
1,1,73,Seamus Pattison,False,True,4116.0,GENERAL,1961.0,Labour Party,0.0954,Elected,1,Carlow Kilkenny,1961 general election - Carlow–Kilkenny,0.57
2,2,74,Seamus Pattison,False,True,6299.0,GENERAL,1965.0,Labour Party,0.1408,Elected,1,Carlow Kilkenny,1965 general election - Carlow–Kilkenny,0.84
3,3,76,Seamus Pattison,False,True,6041.0,GENERAL,1969.0,Labour Party,0.1311,Elected,1,Carlow Kilkenny,1969 general election - Carlow–Kilkenny,0.79
4,4,77,Seamus Pattison,False,True,5300.0,GENERAL,1973.0,Labour Party,0.1134,Elected,1,Carlow Kilkenny,1973 general election - Carlow–Kilkenny,0.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22731,28582,1504,NaN,False,False,309.0,GENERAL,1989.0,Non party/Independent,0.0095,Not Elected,3775,Cork South West,1989 general election - Cork South–West,0.04
22732,28583,103,NaN,False,False,1705.0,GENERAL,1992.0,Non party/Independent,0.0290,Not Elected,4014,Dún Laoghaire,1992 general election - Dún Laoghaire,0.17
22733,28584,104,NaN,False,False,870.0,GENERAL,1997.0,National Party,0.0154,Not Elected,4014,Carlow Kilkenny,1997 general election - Carlow–Kilkenny,0.09
22734,28588,1223,NaN,False,False,219.0,BI-ELECTION,1994.0,Non party/Independent,0.0052,Not Elected,4211,Cork South Central,1994 by-election - Cork South–Central,0.01


In [ ]:
df2.iloc[list(set(df2.index).difference(set(combined_dataframe.df2_index)))]

,election,elected,party,first_pref_pct,first_pref_count,pct_of_quota_reached_with_first_pref,year,candidate,constituency_name,election_type,candidate_ID
5,1985 Local Election - Tullaroan,False,Fianna Fáil,0.121,550,0.61,1985.0,John McGuinness,Tullaroan,LOCAL,
6,1991 Local Election - Tullaroan,True,Fianna Fáil,0.121,633,0.61,1991.0,John McGuinness,Tullaroan,LOCAL,
32793,2016 general election - Wicklow,False,Independent,0.002,112,0.01,2016.0,Bob Kearns,Wicklow,GENERAL,
51,2011 general election - Carlow–Kilkenny,False,Green Party,0.028,2072,0.17,2011.0,Mary White,Carlow–Kilkenny,GENERAL,
52,2011 general election - Carlow–Kilkenny,False,Independent,0.008,601,0.05,2011.0,Stephen Kelly,Carlow–Kilkenny,GENERAL,
...,...,...,...,...,...,...,...,...,...,...,...
32738,1991 Local Election - Wicklow,False,Labour Party,0.038,325,0.23,1991.0,Joseph O'Shaughnessy,Wicklow,LOCAL,
32739,1991 Local Election - Greystones,False,Independent,0.097,679,0.49,1991.0,Jack Murnane,Greystones,LOCAL,
32743,1950 Local Election - Westport,True,Independent,0.176,1530,1.06,1950.0,Joseph Sweeney,Westport,LOCAL,
32744,1955 Local Election - Westport,True,Independent,0.14,1178,0.84,1955.0,Joseph Sweeney,Westport,LOCAL,


In [ ]:
df1[df1['candidate'] == 'John McGuinness']

,year,candidate,candidate_ID,constituency_name,party,elected,election_type,first_pref_count,first_pref_pct,pct_of_quota_reached_with_first_pref,ran_unopposed,status
15,1923.0,John McGuinness,3,Leix Offaly,Republican,True,GENERAL,5572.0,0.1382,None,False,Elected
23719,1997.0,John McGuinness,5481,Carlow Kilkenny,Fianna Fail,True,GENERAL,5990.0,0.1061,0.64,False,Elected
23720,1999.0,John McGuinness,5481,Kilkenny,Fianna Fail,True,LOCAL,1678.0,0.1941,1.55,False,Elected
23721,1999.0,John McGuinness,5481,Kilkenny,Fianna Fail,True,LOCAL,1137.0,0.1646,2.14,False,Elected
23722,2002.0,John McGuinness,5481,Carlow Kilkenny,Fianna Fail,True,GENERAL,9343.0,0.1536,0.92,False,Elected
23723,2007.0,John McGuinness,5481,Carlow Kilkenny,Fianna Fail,True,GENERAL,11635.0,0.172,1.03,False,Elected
23724,2011.0,John McGuinness,5481,Carlow Kilkenny,Fianna Fail,True,None,9531.0,0.1292,0.78,False,Elected
23725,2016.0,John McGuinness,5481,Carlow Kilkenny,Fianna Fail,True,None,10528.0,0.1504,0.9,False,Elected
23726,2020.0,John McGuinness,5481,Carlow Kilkenny,Fianna Fail,True,None,10558.0,0.1434,0.86,False,Elected
24879,1982.0,John McGuinness,8597,Carlow Kilkenny,Fianna Fail,False,GENERAL,907.0,0.0167,None,False,Not Elected


In [ ]:
df1.iloc[list(set(df1.index).difference(set(combined_dataframe.df1_index)))]

,year,candidate,candidate_ID,constituency_name,party,elected,election_type,first_pref_count,first_pref_pct,pct_of_quota_reached_with_first_pref,ran_unopposed,status
15,1923.0,John McGuinness,3,Leix Offaly,Republican,True,GENERAL,5572.0,0.1382,None,False,Elected
30,1997.0,M J Nolan,5,Administrative Panel,Fianna Fail,False,SEANAD,57.0,0.0585,0.47,False,Not Elected
48,2002.0,Mary White,7,Industrial and Commercial Panel,Green/Comhaontas Glas,False,SEANAD,32.0,0.0335,None,False,Not Elected
52,2011.0,Mary White,7,Carlow Kilkenny,Green/Comhaontas Glas,False,None,2072.0,0.0281,0.17,False,Not Elected
53,1999.0,Fergal Browne,8,Carlow,Fine Gael,True,LOCAL,319.0,0.0658,0.66,False,Elected
...,...,...,...,...,...,...,...,...,...,...,...,...
28572,1993.0,Peter Hanna,4144,Industrial and Commercial Panel,Non party/Independent,False,SEANAD,10.0,0.0103,0.1,False,Not Elected
28574,2011.0,John Hyland,6807,Dublin Central,Non party/Independent,False,None,77.0,0.0022,0.01,False,Not Elected
28585,2010.0,William Irwin,4203,Newry and Armagh,Democratic Unionist,False,WESTMINSTER,5764.0,0.1284,None,False,Not Elected
28586,2017.0,William Irwin,4203,Newry and Armagh,Democratic Unionist,False,WESTMINSTER,13177.0,0.2459,None,False,Not Elected


In [ ]:
df1.index

RangeIndex(start=0, stop=28590, step=1)

In [ ]:
combined_dataframe.df1_index.isin(df1.index)

0        True
1        True
2        True
3        True
4        True
         ... 
22731    True
22732    True
22733    True
22734    True
22735    True
Name: df1_index, Length: 22736, dtype: bool

In [ ]:
df1[~combined_dataframe.df1_index.isin(df1.index)]

C:\Users\Rober\AppData\Local\Temp\ipykernel_19388\3962292584.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1[~combined_dataframe.df1_index.isin(df1.index)]


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [ ]:
# is the year the same
# and is the data the same

for row in t1.fillna('-').itertuples():
    print('row',row.constituency_name)
    similar_name_same_year_df = t2[(t2.year == row.year)]
    if similar_name_same_year_df.shape[0] > 1: # if there are multiple people with similar names running in the same year
        # check constituency name similarity
        most_similar = ''
        highest_jaro_winler = 0

        for constituency in similar_name_same_year_df.constituency_name:
            if jellyfish.jaro_winkler(constituency,row.constituency_name) >= highest_jaro_winler:
                most_similar = constituency
                highest_jaro_winler = jellyfish.jaro_winkler(constituency,row.constituency_name)

        print('m:',most_similar)
        #similar_name_same_year_df
        matched_row = similar_name_same_year_df[similar_name_same_year_df['constituency_name'] == most_similar]

    else: # we have 1 row matched with 1 row in another dataframe. so combine them
        #print('beep')
        matched_row = similar_name_same_year_df[0]
        duno = row._asdict()
        ddos = list(matched_row.itertuples())._asdict()
        d =  create_new_dictionary(duno,ddos)
        print(d)
        print('-------------------------')
        

In [ ]:
t1

In [ ]:
similar_name_same_year_df

In [ ]:
similar_name_same_year_df.shape

In [ ]:
temp_df.head()

In [ ]:
temp_df2.head()

Run through and say per name in 